In [2]:
import requests
import re
import pandas as pd
from google.colab import drive
from bs4 import BeautifulSoup as bs4
import gspread

# Proses mengambil HTML code dari Bucaka.com
url = 'https://bucaka.com/'

try:
    response_url = requests.get(url).text

except requests.exceptions.RequestException as e:
    print(f"Error: {e}")

# Proses memotong HTML ke data yang diinginkan
response_filter_1 = bs4(response_url,"lxml")
response = response_filter_1.find_all('div', {"class":"load-more-location col-sm-6 col-12 col-lg-3 mb-6 mb-lg-5"})

# Proses menghilangkan HTML code yang tidak dibutuhkan
unwanted_words = ['<div class="load-more-location col-sm-6 col-12 col-lg-3 mb-6 mb-lg-5"><div class="pt-4 pb-4 pb-1 px-4 d-flex flex-column bg-white text-center h-100 ag-courses text-sm-start text-start rounded-3 shadow"><div class="ag-courses-item_bg"></div><h5 class="__className_b5b299" style="text-transform:capitalize;font-size:undefinedpx">','<!-- --> </h5><p class="work1 mt-auto mb-0 __className_b5b299" style="font-size:16px">','</p></div></div>']
response_filter_2 = [response.text for response in response if response.text not in unwanted_words]

# Proses menghilangkan simbol titik pada angka dan text kandidat
modified_list = [item.replace(".", "") for item in response_filter_2]
modified_list_2 = [item.replace("kandidat", "") for item in modified_list]

# Proses memisahkan text dan angka
provinsi = []
jumlah_kandidat = []

for item in modified_list_2:
    match = re.match(r'([a-zA-Z\s]+)(\d+)', item)
    if match:
        text, number = match.groups()
        provinsi.append(text.strip())
        jumlah_kandidat.append(int(number))

# Proses membuat DataFrame tanpa nomor urut
data = { 'Provinsi': provinsi, 'Kandidat': jumlah_kandidat}
df = pd.DataFrame(data)

# Menampilkan data dalam data frame
df

# Mount Google Drive
drive.mount('/content/gdrive')

# Ganti '/content/gdrive/MyDrive/nama_folder/nama_file.xlsx' sesuai dengan path dan nama file Excel di Google Drive
file_path_drive_excel = '/content/gdrive/MyDrive/bucakascraping2.xlsx'

# Simpan DataFrame ke file Excel
df.to_excel(file_path_drive_excel, index=False)


,Provinsi,Kandidat
0,BALI,644
1,BANGKA BELITUNG,38
2,BANTEN,1572
3,BENGKULU,41
4,DI YOGYAKARTA,951
5,DKI JAKARTA,9281
6,GORONTALO,27
7,JAMBI,118
8,JAWA BARAT,4737
9,JAWA TENGAH,1623


from matplotlib import pyplot as plt
df['Kandidat'].plot(kind='hist', bins=20, title='Kandidat')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
df['Kandidat'].plot(kind='line', figsize=(8, 4), title='Kandidat')
plt.gca().spines[['top', 'right']].set_visible(False)